# Нейронная сеть для проекта

## Обработка данных

In [3]:
import tensorflow_datasets

from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE

d:\roman\Documents\Study\Major\Junior\Semester_2\Интеллектуальная обработка данных\Проект\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def tokenize(x, tokenizer):
    return ' '.join(tokenizer.tokenize(x.lower()))

In [5]:
# загружаем данные и токенизатор
data = tensorflow_datasets.load('ted_hrlr_translate/ru_to_en')
tokenizer = WordPunctTokenizer()

# разбиваем и токенизируем тексты, записываем обработанные токены в файл
for lang in ['en', 'ru']:
    with open('../data/train.' + lang, 'w', encoding='utf-8') as f_out:
        for line in data['train']:
            src_line = line[lang].numpy().decode('utf8')
            f_out.write(tokenize(src_line, tokenizer) + '\n')

# строим и применяем bpe кодирование
bpe = {}
for lang in ['en', 'ru']:
    learn_bpe(open('../data/train.' + lang, encoding='utf-8'), open('../data/bpe_rules.' + lang, 'w', encoding='utf-8'), num_symbols=8000)
    bpe[lang] = BPE(open('../data/bpe_rules.' + lang, encoding='utf-8'))
    
    with open('../data/train.bpe.' + lang, 'w', encoding='utf-8') as f_out:
        for line in open('../data/train.' + lang, encoding='utf-8'):
            f_out.write(bpe[lang].process_line(line.strip()) + '\n')

100%|██████████| 8000/8000 [00:33<00:00, 236.50it/s] 


## Загрузка данных

In [13]:
import numpy as np

from vocab import Vocab
from sklearn.model_selection import train_test_split

In [15]:
data_inp = np.array(open('../data/train.bpe.ru', encoding='utf-8').read().split('\n'))
data_out = np.array(open('../data/train.bpe.en', encoding='utf-8').read().split('\n'))

train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=0.3, random_state=0)

In [16]:
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')

inp: не поступа@@ йте так !
out: do not do that .

inp: ( смех в зале ) а потом она на меня посмотре@@ ла .
out: ( laughter ) and then she looked at me .

inp: в центре g@@ al@@ l@@ u@@ p я изуча@@ ю мусуль@@ ман@@ ские сообщества во всем мире .
out: i study muslim societies around the world at gall@@ up .



In [30]:
import pickle

inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

with open('../data/voc.ru', 'wb') as file:
    pickle.dump(inp_voc, file)
with open('../data/voc.en', 'wb') as file:
    pickle.dump(out_voc, file)

In [18]:
# тут можно посмотреть, как работает мапинг из индекса в токен и наоборот
batch_lines = train_inp[:3]
batch_ids = inp_voc.to_matrix(batch_lines)
batch_lines_restored = inp_voc.to_lines(batch_ids)

print("lines")
print(batch_lines)
print("\nwords to ids (0 = bos, 1 = eos):")
print(batch_ids)
print("\nback to words")
print(batch_lines_restored)

lines
['не поступа@@ йте так !'
 '( смех в зале ) а потом она на меня посмотре@@ ла .'
 'в центре g@@ al@@ l@@ u@@ p я изуча@@ ю мусуль@@ ман@@ ские сообщества во всем мире .']

words to ids (0 = bos, 1 = eos):
tensor([[   0, 3805, 5006, 2591, 6816,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   0,   11, 6255,  656, 2141,   12,  297, 5021, 4285, 3621, 3366, 4984,
         3003,   19,    1,    1,    1,    1,    1,    1],
        [   0,  656, 7610,  156,  121,  183,  237,  206, 8124, 2401, 8090, 3593,
         3284, 6136, 6405,  976, 1127, 3451,   19,    1]])

back to words
['не поступа@@ йте так !', '( смех в зале ) а потом она на меня посмотре@@ ла .', 'в центре g@@ al@@ l@@ u@@ p я изуча@@ ю мусуль@@ ман@@ ские сообщества во всем мире .']


## Обучение модели

In [19]:
import torch
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm
from functional import loss_function, compute_bleu
from model import AttentiveModel

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
metrics = {'train_loss': [], 'dev_bleu': [] }

model = AttentiveModel(inp_voc, out_voc).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
batch_size = 32

num_iter = 10000

for i in tqdm(range(num_iter)):
    batch_indices = np.random.randint(len(train_inp), size=batch_size)
    batch_lines_inp = train_inp[batch_indices]
    batch_lines_out = train_out[batch_indices]
    batch_inp = inp_voc.to_matrix(batch_lines_inp).to(device)
    batch_out = out_voc.to_matrix(batch_lines_out).to(device)

    opt.zero_grad()
    loss = loss_function(model, batch_inp, batch_out)
    loss.backward()
    opt.step()
    metrics['train_loss'].append((i, loss.item()))
    if i % 100 == 0:
        bleu = compute_bleu(model, batch_lines_inp, batch_lines_out)
        metrics['dev_bleu'].append((i, bleu))

  0%|          | 8/10000 [00:26<9:18:30,  3.35s/it] 


KeyboardInterrupt: 

In [ ]:
for inp_line, trans_line in zip(dev_inp[:3], model.translate_lines(dev_inp[:3])[0]):
    print(inp_line, trans_line, sep='\n', end='\n\n')

для этого не нужно быть полностью сле@@ пы@@ м , достаточно суще@@ ственного расстрой@@ ства зрения .
, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

но несмотря на то , что мы узнали о болезни сравни@@ тельно недавно и не владе@@ ем ис@@ чер@@ пы@@ ва@@ ющей информацией , мы знаем , как её остановить .
, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

« о чём ты , эли@@ н ? » « да ни о чём особен@@ ном . о том , о с@@ ём , о ра@@ е и а@@ де . давайте подни@@ ме@@ мся на кры@@ шу .
, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

